In [32]:
import pandas as pd
import edgar_functions as ef
import datetime

The following three functions extract the cashflow statements from EDGAR, filter and transform them into a more readable form and finally combine them into a single table that can be used for further analysis.

In [33]:
def extract_cash(ticker):
   
    #extract
    acc = ef.get_filtered_filings(ticker, ten_k=True, just_accession_numbers=True, headers=ef.headers)
    acc_num = acc.iloc[0].replace('-', '') 
    data = ef.process_one_statement(ticker, acc_num, 'cash_flow_statement') # fetches the updated cash flow statement
   
    #raw transformation
    current_year = datetime.datetime.now().year
    data.reset_index(inplace=True)
    data.columns = ['transaction', current_year - 1, current_year - 2, current_year -3]
    data['transaction'] = data['transaction'].str.strip('us-gaap_')

    return data

In [34]:
def transform_cash_flow(ticker):
    
    raw_cash_data = extract_cash(ticker)
    desired_rows = ['NetIncomeLo', 'NetCashProvidedByUsedInOperatingActivitie', 'NetCashProvidedByUsedInInvestingActivitie']
    subset = raw_cash_data[raw_cash_data['transaction'].isin(desired_rows)] # filtering the rows of interest
    subset.set_index('transaction', inplace=True) # set index to transaction while transposing the table

    df = subset.T
    df.reset_index(inplace=True) # turn 'transaction' back to a regular column

    columns = ['year', 'earnings', 'cash_from_operations', 'net_cash_from_investments']
    df.columns = columns # rename the columns to a more readable form

    df['owners_earnings'] = df['cash_from_operations'] + df['net_cash_from_investments']
    df['company_ticker'] = ticker
    
    return df

In [28]:
def collected_cash_data(ticker_list):
    """This function takes a list of company_tickers and fetches the cash flow dataframes using those tickers,
        concatenating the tables and returning it for further analysis""" 
    
    dfs = []

    for ticker in ticker_list:
        df = transform_cash_flow(ticker)
        dfs.append(df)

    concat_df = pd.concat(dfs, axis=0)
    return concat_df

Testing: 

In [29]:
ticker_list = ['AAPL', 'AMZN', 'MSFT', 'GOOG']

In [30]:
collected_data = collected_cash_data(ticker_list)

In [31]:
collected_data

,year,earnings,cash_from_operations,net_cash_from_investments,owners_earnings,company_ticker
0,2023,"96,995,000","110,543,000","3,705,000","114,248,000",AAPL
1,2022,"99,803,000","122,151,000","-22,354,000","99,797,000",AAPL
2,2021,"94,680,000","104,038,000","-14,545,000","89,493,000",AAPL
0,2023,"30,425,000","84,946,000","-49,833,000","35,113,000",AMZN
1,2022,"-2,722,000","46,752,000","-37,601,000","9,151,000",AMZN
2,2021,"33,364,000","46,327,000","-58,154,000","-11,827,000",AMZN
0,2023,"72,361,000","87,582,000","-22,680,000","64,902,000",MSFT
1,2022,"72,738,000","89,035,000","-30,311,000","58,724,000",MSFT
2,2021,"61,271,000","76,740,000","-27,577,000","49,163,000",MSFT
0,2023,"73,795,000","101,746,000","-27,063,000","74,683,000",GOOG
